#  Regression on Diamonds Price Dataset with SVM

The **Diamonds dataset** from Kaggle is a dataset containing information about the physical and pricing attributes of nearly 54,000 diamonds. The dataset is commonly employed in tasks like regression analysis, feature engineering, and exploratory data analysis.

We will consider a **reduced version** of the dataset, containing 4000 samples, and without categorical features.

### Key Features:
- **Carat**: The weight of the diamond.
- **Depth**: The total depth percentage (z / mean(x, y)).
- **Table**: Width of the diamond's top as a percentage of its widest point.
- **Price**: Price in US dollars.
- **X, Y, Z**: Dimensions of the diamond in mm (length, width, depth).

This dataset is useful for exploring relationships between physical attributes and pricing, and for building predictive models to estimate diamond prices based on their features.

For more information see: https://www.kaggle.com/datasets/shivam2503/diamonds.

# Overview

In the notebook you will perform a complete pipeline of machine learning - regression task. First, you will:
- split the data into training, validation, and test;
- standardize the data.

You will then be asked to learn various SVM models, in particular:
- for each of the kernels *linear*, *poly*, *rbf*, and *sigmoid*, you will learn the best model, choosing among some fixed values of the considered hyperparameters. In particular, the choice of hyperparameters must be done with **5-fold cross-validation**, as we have seen in the labs.

Then, from the models trained with the best hyperparameters selected as above, you will:
- choose the best kernel, using a validation approach (not cross-validation), and
- learn the best SVM model overall.

Furthermore, you will then be asked to estimate the generalization error of the best SVM model you report.

At the end, just for comparison, you will also be asked to learn a standard linear regression model (with squared loss), and estimate its generalization error.

### IMPORTANT
- Note that in each of the above steps you will have to choose the appropriate split of the data (see the first bullet point above);
- The code should run without requiring modifications even if some best choice of parameters, changes; for example, you should not pass the best value of hyperparameters "manually" (i.e., passing the values as input parameters to the models). The only exception is in the TO DO titled 'ANSWER THE FOLLOWING'
- $\texttt{epsilon}$ parameter: For SVM, since the values to be predicted are all in the thousands of dollars, you will need to always set $\texttt{epsilon} = 100$
- Do not change the printing instructions (other than adding the correct variable name for your code), and do not add printing instructions!

## TO DO - INSERT YOUR NUMERO DI MATRICOLA BELOW

In [11]:
# -- put here your ID number (numero di matricola)
numero_di_matricola = 2148370 # Spicoli Piersilvio

The following code loads all required packages

In [12]:
# -- import all packages needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn import model_selection
from sklearn import linear_model
from sklearn.model_selection import KFold
from itertools import product

The code below loads the data and remove samples with missing values. It also prints the number of samples and a brief description of our dataset.

In [13]:
# -- load the data - do not change the path below!
df = pd.read_csv('diamonds.csv', sep = ',')

# -- remove the data samples with missing values (NaN)
df = df.dropna()
# -- let's drop the column containing the id of the data
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [14]:
print('Dataset shape:', df.shape)
# -- description of dataset
print(df.describe())

Dataset shape: (4000, 7)
             carat        depth       table         price            x  \
count  4000.000000  4000.000000  4000.00000   4000.000000  4000.000000   
mean      0.797945    61.776925    57.44035   3920.239250     5.735810   
std       0.462251     1.468899     2.26052   3935.292841     1.106897   
min       0.210000    52.200000    52.00000    339.000000     0.000000   
25%       0.400000    61.100000    56.00000    936.000000     4.720000   
50%       0.710000    61.900000    57.00000   2468.000000     5.710000   
75%       1.050000    62.500000    59.00000   5297.500000     6.550000   
max       3.010000    70.600000    79.00000  18730.000000     9.100000   

                 y            z  
count  4000.000000  4000.000000  
mean      5.736307     3.540002  
std       1.099129     0.691834  
min       0.000000     0.000000  
25%       4.730000     2.910000  
50%       5.730000     3.540000  
75%       6.550000     4.040000  
max       8.970000     5.670000  


In [15]:
print('First 5 samples of the dataset:\n\n', df.head(5))

First 5 samples of the dataset:

    carat  depth  table  price     x     y     z
0   0.33   61.7   55.0    564  4.43  4.46  2.74
1   1.20   62.1   57.0   5914  6.78  6.71  4.19
2   0.62   61.0   57.0   2562  5.51  5.54  3.37
3   0.34   63.1   56.0    537  4.41  4.46  2.80
4   1.20   62.5   55.0   5964  6.77  6.84  4.25


In the following cell, we convert our (pandas) dataframe into set X (containing our features) and the set Y (containing our target, i.e., the price)

In [16]:
m = df.shape[0]

# -- let's compute X and Y sets
X = df.drop(columns=['price'], axis=1)
Y = df['price']

print("Total number of samples:", m)

X = X.values
Y = Y.values

# -- print shapes
print('X shape: ', X.shape)
print('Y shape: ', Y.shape)

Total number of samples: 4000
X shape:  (4000, 6)
Y shape:  (4000,)


# Data preprocessing

## TO DO - SPLIT DATA INTO TRAINING, VALIDATION, AND TESTING, WITH THE FOLLOWING PERCENTAGES: 60%, 20%, 20%

Use the $\texttt{train\_test\_split}$ function from sklearn.model_selection to do it; in every call fix $\texttt{random\_state}$ to your numero_di_matricola.
At the end, you should store the data in the following variables:
- X_train, Y_train: training data;
- X_val, Y_val: validation data;
- X_train_val, Y_train_val: training and validation data;
- X_test, Y_test: test data.

The code then prints the number of samples in X_train, X_val, X_train_val, and X_test

**IMPORTANT:**
- first split the data into training+validation and test; the first part of the data in output from $\texttt{train\_test\_split}$ must correspond to the training+validation;
- then split training+validation into training and validation; the first part of the data in output from $\texttt{train\_test\_split}$ must correspond to the training


In [17]:
# -- split the data into training + validation and test

m_test = int(X.shape[0] * 0.2)
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size = m_test, random_state = numero_di_matricola)

# -- split the training + validation data into training and validation

X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = m_test, random_state = numero_di_matricola)

print("Training size:", X_train.shape[0])
print("Validation size:", X_val.shape[0])
print("Training and validation size:", X_train_val.shape[0])
print("Test size:", X_test.shape[0])

Training size: 2400
Validation size: 800
Training and validation size: 3200
Test size: 800


## TO DO - STANDARDIZE THE DATA

Standardize the data using the $\texttt{preprocessing.StandardScaler}$ from scikit learn.

If V is the name of the variable storing part of the data, the corresponding standardized version should be stored in V_scaled. For example, the scaled version of X_train should be stored in X_train_scaled.

In [18]:
s = preprocessing.StandardScaler()
s.fit(X_train)
X_train_scaled = s.transform(X_train)
X_train_val_scaled = s.transform(X_train_val)
X_val_scaled = s.transform(X_val)
X_test_scaled = s.transform(X_test)

print("X_train_scaled: mean = ", np.mean(X_train_scaled, axis=0))
print("X_train_scaled: standard dev = ", np.std(X_train_scaled, axis=0))

print("X_train_val_scaled: mean = ", np.mean(X_train_val_scaled, axis=0))
print("X_train_val_scaled: standard dev = ", np.std(X_train_val_scaled, axis=0))

print("X_val_scaled: mean = ", np.mean(X_val_scaled, axis=0))
print("X_val_scaled: standard dev = ", np.std(X_val_scaled, axis=0))

print("X_test_scaled: mean = ", np.mean(X_test_scaled, axis=0))
print("X_test_scaled: standard dev = ", np.std(X_test_scaled, axis=0))

X_train_scaled: mean =  [ 1.06563947e-14  6.73724994e-14 -4.63937916e-15 -9.38361657e-15
 -5.64099115e-15  2.22859347e-15]
X_train_scaled: standard dev =  [1. 1. 1. 1. 1. 1.]
X_train_val_scaled: mean =  [-0.0008297  -0.00664266  0.00203145 -0.000272   -0.00023556  0.00074503]
X_train_val_scaled: standard dev =  [1.00314206 0.99307292 0.9999866  1.00024937 1.00007576 0.99504154]
X_val_scaled: mean =  [-0.00331878 -0.02657065  0.00812579 -0.00108799 -0.00094226  0.00298013]
X_val_scaled: standard dev =  [1.01250567 0.97172313 0.99992165 1.00099665 1.00030267 0.98001226]
X_test_scaled: mean =  [-0.00773776 -0.01295922  0.0136107  -0.01789684 -0.01678134 -0.01149391]
X_test_scaled: standard dev =  [1.03013979 1.02511242 1.00893552 1.03318451 1.03015919 1.007204  ]


# SVM models: learning the best model for each kernel

The following function, i.e., $\texttt{k\_fold\_cross\_validation}$, will perform $k$-fold cross validation (with $k$ = 5 by default). Look carefully at the signature of the below function: you have in input some sets X and Y, the default number of folds, and a length-variable keyword argumens, with which the SVM model will be trained in the cross-validation phase. If you are not familiar with the notation, look at kwargs in Python documentation.

In the first lines of the below function, the unpacked parameters (i.e., input parameter $\texttt{param\_grid}$) are converted into a python list by means of cartesian product. The resulting list (i.e., $\texttt{param\_list}$) will be the one for which you need to iterate over and perform $k$-fold cross-validation, using $\texttt{KFold}$ object frmo scikit-learn.

At the end, note that you need to return $\texttt{best\_param}$, that is the best set of parameters you found with the cross-validation procedure.

In [19]:
from sklearn.svm import SVC

def k_fold_cross_validation(X, Y, num_folds = 5, **param_grid):

    # -- grid of hyperparams into list
    param_keys = list(param_grid.keys())
    param_values = list(param_grid.values())
    
    # Generate Cartesian product of values
    combinations = product(*param_values)
    
    # Create a list of dictionaries from combinations
    param_list = [dict(zip(param_keys, combination)) for combination in combinations]

    # -- TODO
    kf = KFold(n_splits = num_folds)
    err_train_kfold = np.zeros(len(param_list))
    err_val_kfold = np.zeros(len(param_list))
    
    for i, param in enumerate(param_list):
        
        model = svm.SVR(**param)
        
        for train_index, validation_index in kf.split(X):
            X_train_kfold, X_val_kfold = X[train_index], X[validation_index]
            Y_train_kfold, Y_val_kfold = Y[train_index], Y[validation_index]
            
            scaler_kfold = preprocessing.StandardScaler().fit(X_train_kfold)
            X_train_kfold_scaled = scaler_kfold.transform(X_train_kfold)
            X_val_kfold_scaled = scaler_kfold.transform(X_val_kfold)
            
            model.fit(X_train_kfold_scaled, Y_train_kfold)
            Y_pred_train = model.predict(X_train_kfold_scaled)
            Y_pred_val = model.predict(X_val_kfold_scaled)

            err_train_kfold[i] += (1 - model.score(X_train_kfold_scaled, Y_train_kfold))
            err_val_kfold[i] += (1 - model.score(X_val_kfold_scaled, Y_val_kfold))

    err_train_kfold /= num_folds
    err_val_kfold /= num_folds
    best_param = param_list[np.argmin(err_val_kfold)]
    
    return best_param
    

## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR LINEAR KERNEL

For the SVM, consider $\texttt{svm.SVR}$ class. We will begin by training the SVM with linear kernel. For the latter, consider the following hyperparameters and their values:

- $C: [0.1, 1, 10, 100, 1000]$

Remember that both the $\texttt{kernel}$ type and the value of $\texttt{epsilon}$ are considered as parameters to pass to the above method. Leave all other input parameters to default.

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [20]:
print("\nLinear SVM:")

linear_param_grid = {
    "C": [0.1, 1, 10, 100, 1000],
    "epsilon": [100],
    "kernel": ["linear"]
}

best_param_linear = k_fold_cross_validation(X_train_scaled, Y_train, num_folds=5, **linear_param_grid)

print("Best value for hyperparameters: ", best_param_linear)


Linear SVM:
Best value for hyperparameters:  {'C': 1000, 'epsilon': 100, 'kernel': 'linear'}


## TO DO - LEARN A MODEL WITH LINEAR KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [21]:
linear_model = svm.SVR(**best_param_linear)
linear_model.fit(X_train_scaled, Y_train)

print("Training score:", linear_model.score(X_train_scaled, Y_train))

Training score: 0.8420874176798536


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR POLY KERNEL

Now, let's consider $\texttt{svm.SVR}$ with polynomial kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $degree: [2, 3, 4]$

Leave all other input parameters to default.

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [22]:
print("\nPoly SVM")

param_grid_poly = {
    "C": [0.1, 1, 10, 100, 1000],
    "kernel": ["poly"],
    "epsilon": [100],
    "degree": [2, 3, 4]
}

best_param_poly = k_fold_cross_validation(X_train_scaled, Y_train, num_folds=5, **param_grid_poly)

print("Best value for hyperparameters: ", best_param_poly)


Poly SVM
Best value for hyperparameters:  {'C': 10, 'kernel': 'poly', 'epsilon': 100, 'degree': 3}


## TO DO - LEARN A MODEL WITH POLY KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [23]:
poly_model = svm.SVR(**best_param_poly)
poly_model.fit(X_train_scaled, Y_train)

print("Training score:", poly_model.score(X_train_scaled, Y_train))

Training score: 0.47421265911874444


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR RBF KERNEL

Consider $\texttt{svm.SVR}$ with RBF kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $gamma: [0.01, 0.03, 0.04, 0.05]$

Leave all other input parameters to default.

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [24]:
print("\nRBF SVM")

param_grid_RBF = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [0.01, 0.03, 0.04, 0.05],
    "epsilon": [100],
    "kernel": ["rbf"]
}

best_param_RBF = k_fold_cross_validation(X_train_scaled, Y_train, num_folds=5, **param_grid_RBF)

print("Best value for hyperparameters: ", best_param_RBF)


RBF SVM
Best value for hyperparameters:  {'C': 1000, 'gamma': 0.04, 'epsilon': 100, 'kernel': 'rbf'}


## TO DO - LEARN A MODEL WITH RBF KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [25]:
RBF_model = svm.SVR(**best_param_RBF)
RBF_model.fit(X_train_scaled, Y_train)

print("Training score:", RBF_model.score(X_train_scaled, Y_train))

Training score: 0.860381286714856


## TO DO - CHOOSE THE BEST HYPERPARAMETERS FOR SIGMOID KERNEL

Consider $\texttt{svm.SVR}$ with sigmoid kernel. Consider the following hyperparameters and their values:
- $C: [0.1, 1, 10, 100, 1000]$
- $gamma: [0.01, 0.05, 0.1]$
- $coef0: [0, 1]$

Leave all other input parameters to default.

Find the best value of the hyperparameters using 5-fold cross validation. Use the function defined above to perform the cross-validation.

Print the best value of the hyperparameters.

In [26]:
print("\nSigmoid SVM")

param_grid_sig = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [0.01, 0.05, 0.1],
    "kernel": ["sigmoid"],
    "epsilon": [100],
    "coef0": [0, 1]
}

best_param_sig = k_fold_cross_validation(X_train_scaled, Y_train, num_folds=5, **param_grid_sig)

print("Best value for hyperparameters: ", best_param_sig)


Sigmoid SVM
Best value for hyperparameters:  {'C': 1000, 'gamma': 0.01, 'kernel': 'sigmoid', 'epsilon': 100, 'coef0': 0}


## TO DO - LEARN A MODEL WITH SIGMOID KERNEL AND BEST CHOICE OF HYPERPARAMETERS

This model will be compared with the best models with other kernels using validation (not cross validation).

DO NOT PASS PARAMETERS BY HARD-CODING THEM IN THE CODE.

Print the **training score** (that is, $R^2$ coefficient) of the best model, trained with the best parameter find from the above cell.

In [27]:
sig_model = svm.SVR(**best_param_sig)
sig_model.fit(X_train_scaled, Y_train)

print("Training score:", sig_model.score(X_train_scaled, Y_train))

Training score: 0.7783920869877037


## TO DO - USE VALIDATION TO CHOOSE THE BEST MODEL AMONG THE ONES LEARNED FOR THE VARIOUS KERNELS

Use validation to choose the best model among the four ones (one for each kernel) you have learned above.

Print, following exactly the order described here, with 1 value for each line:
- the validation score of SVM with linear kernel (the template below does not include such print)
- the validation score of SVM with polynomial kernel (the template below does not include such print)
- the validation score of SVM with rbf kernel (the template below does not include such print)
- the validation score of SVM with sigmoid kernel (the template below does not include such print)
- the best kernel (e.g., sigmoid)
- the validation score of the best kernel

For the first 4 prints, use the format: "*kernel* validation score: ". For example, for linear kernel "linear validation score: ", for rbf "rbf validation score: "

In [28]:
print("\nVALIDATION TO CHOOSE SVM KERNEL:")

validation_scores = {
    "linear": linear_model.score(X_val_scaled, Y_val),
    "polynomial": poly_model.score(X_val_scaled, Y_val),
    "rbf": RBF_model.score(X_val_scaled, Y_val),
    "sigmoid": sig_model.score(X_val_scaled, Y_val)
}

main_params_keys = {
    "linear": ["C"],
    "polynomial": ["C", "degree", "gamma"],
    "rbf": ["C", "gamma"],
    "sigmoid": ["C", "gamma"]
}

grid_params = {
    "linear": {key: linear_model.get_params()[key] for key in main_params_keys["linear"]},
    "polynomial": {key: poly_model.get_params()[key] for key in main_params_keys["polynomial"]},
    "rbf": {key: RBF_model.get_params()[key] for key in main_params_keys["rbf"]},
    "sigmoid": {key: sig_model.get_params()[key] for key in main_params_keys["sigmoid"]}
}

best_kernel = max(validation_scores, key=validation_scores.get)
best_score = validation_scores[best_kernel]
best_params = grid_params[best_kernel]

for kernel, score in validation_scores.items():
    print(f"{kernel} validation score: {score:.4f}")

print("\n---")
print(f"Best kernel: {best_kernel}")
print(f"Best parameters for the best kernel: {best_params}")
print(f"Validation score of best kernel: {best_score:.4f}")



VALIDATION TO CHOOSE SVM KERNEL:
linear validation score: 0.8282
polynomial validation score: 0.4519
rbf validation score: 0.8651
sigmoid validation score: 0.7981

---
Best kernel: rbf
Best parameters for the best kernel: {'C': 1000, 'gamma': 0.04}
Validation score of best kernel: 0.8651


## TO DO - LEARN THE FINAL MODEL FOR WHICH YOU WANT TO ESTIMATE THE GENERALIZATION SCORE

Learn the final model (i.e., the one you would use to make predictions about future data).

Print the **final model hyperparameters** and the **score** of the model on the data used to learn it.

In [29]:
print("\nBEST MODEL:")

final_model = svm.SVR(**best_params)
final_model.fit(X_train_val_scaled, Y_train_val)

train_score = final_model.score(X_train_val_scaled, Y_train_val)

print("Best model hyperparameters:", best_params)
print("Score of the best model on the data used to learn it: ", train_score)


BEST MODEL:
Best model hyperparameters: {'C': 1000, 'gamma': 0.04}
Score of the best model on the data used to learn it:  0.8610769454259236


## TO DO - PRINT THE ESTIMATE  OF THE GENERALIZATION SCORE FOR THE FINAL MODEL

Print the estimate of the generalization **score** for the final model. The generalization "score" is the score computed on the data used to estimate the generalization error.

In [30]:
print("\nGENERALIZATION SCORE BEST MODEL:")

final_model = svm.SVR(**best_params)  
final_model.fit(X_train_val_scaled, Y_train_val)  

test_score = final_model.score(X_test_scaled, Y_test) 

print("Best model hyperparameters:", best_params)
print(f"Generalization score (R²) of the best model on the test data: {test_score:.4f}")


GENERALIZATION SCORE BEST MODEL:
Best model hyperparameters: {'C': 1000, 'gamma': 0.04}
Generalization score (R²) of the best model on the test data: 0.8839


## TO DO - ANSWER THE FOLLOWING

Print the **training score** (score on data used to train the model) and the **generalization score** (score on data used to assess generalization) of the final SVM model THAT YOU OBTAIN WHEN YOU RUN THE CODE, one per line, printing the smallest one first.

NOTE: THE VALUES HERE SHOULD BE HARDCODED.

Print you answer (YES/NO) to the following question: does the relation (i.e., smaller, larger) between the training score and the generalization score agree with the theory?

Print your motivation for the YES/NO answer above, using at most 500 characters.

In [31]:
print("\nANSWER")

# -- note that you may have to invert the order of the following 2 lines, print the smallest 1 first
print("Training score: ", train_score)
print("Generalization score: ", test_score)


# -- the following is a string with you answer
motivation = "According to the theory, the obtained results are not consistent because, conceptually, the error on the training data should be slightly higher than the generalization error, and this behavior is due to excessive regularization in the training phase."

print(motivation)


ANSWER
Training score:  0.8610769454259236
Generalization score:  0.8839112859838582
According to the theory, the obtained results are not consistent because, conceptually, the error on the training data should be slightly higher than the generalization error, and this behavior is due to excessive regularization in the training phase.


## TO DO: LEARN A STANDARD LINEAR MODEL
Learn a standard linear model using scikit learn.

Print the **score** of the model on the data used to learn it.

Print the **generalization score** of the model.

In [32]:
from sklearn.linear_model import LinearRegression

print("\nLR MODEL")

linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_val_scaled, Y_train_val)

print("Score of LR model on data used to learn it: ", linear_regression_model.score(X_train_val_scaled, Y_train_val))
print("Generalization score of LR model: ", linear_regression_model.score(X_test_scaled, Y_test))


LR MODEL
Score of LR model on data used to learn it:  0.8488909277502004
Generalization score of LR model:  0.8651633433441661
